<a href="https://colab.research.google.com/github/letsgoahasanur/LRScheduler/blob/main/lrscheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import time
import psutil


In [73]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [74]:
def get_model(model_name, num_classes=10):
    if model_name.lower() == "mobilenet":
        model = models.mobilenet_v2(weights=None)
        model.classifier[1] = nn.Linear(model.last_channel, num_classes)
    elif model_name.lower() == "efficientnet":
        model = models.efficientnet_b0(weights=None)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    else:
        raise ValueError("Model not supported. Choose 'mobilenet' or 'efficientnet'.")

    return model.to(device)


In [75]:
# Choose your model here
#model = get_model("mobilenet")      # MobileNetV2
model = get_model("efficientnet")     # EfficientNet-B0



In [76]:
# Data transformations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])

# Download and load dataset
train_dataset = datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform_train
)

test_dataset = datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform_test
)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# --- Check DataLoader ---
print("CIFAR-10 dataset loaded successfully!")
print("Train batches:", len(train_loader))
print("Test batches:", len(test_loader))

images, labels = next(iter(train_loader))
print("One batch - Images shape:", images.shape, "Labels shape:", labels.shape)


CIFAR-10 dataset loaded successfully!
Train batches: 391
Test batches: 79
One batch - Images shape: torch.Size([128, 3, 32, 32]) Labels shape: torch.Size([128])


In [77]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning Rate Scheduler - StepLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=20,    # total epochs
    eta_min=0.00001
)

print("Loss, optimizer, and StepLR scheduler ready!")


Loss, optimizer, and StepLR scheduler ready!


In [78]:
from sklearn.metrics import precision_score, recall_score, f1_score
import psutil
import torch.cuda as cuda

def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, epochs=20):
    # Lists to store metrics
    train_acc_list, val_acc_list = [], []
    precision_list, recall_list, f1_list = [], [], []
    epoch_times = []
    cpu_usage_list, gpu_mem_list = [], []

    for epoch in range(epochs):
        start_time = time.time()
        model.train()
        correct, total, running_loss = 0, 0, 0

        # CPU usage at start
        cpu_start = psutil.cpu_percent(interval=None)
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Step scheduler if StepLR or CosineAnnealing
        if isinstance(scheduler, torch.optim.lr_scheduler._LRScheduler):
            scheduler.step()

        train_acc = 100 * correct / total
        train_acc_list.append(train_acc)

        # Validation
        model.eval()
        correct, total = 0, 0
        all_labels, all_preds = [], []

        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())

        val_acc = 100 * correct / total
        val_acc_list.append(val_acc)

        # Metrics
               # Metrics
        precision = precision_score(
            all_labels, all_preds, average='macro', zero_division=0
        )
        recall = recall_score(
            all_labels, all_preds, average='macro', zero_division=0
        )
        f1 = f1_score(
            all_labels, all_preds, average='macro', zero_division=0
        )


        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        # Resource usage
        epoch_time = time.time() - start_time
        epoch_times.append(epoch_time)
        cpu_usage_list.append(psutil.cpu_percent(interval=None))
        gpu_mem_list.append(cuda.memory_allocated(device)/1024**2)  # in MB

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}% | "
              f"Precision: {precision:.2f} | Recall: {recall:.2f} | F1: {f1:.2f} | "
              f"Time: {epoch_time:.2f}s")

    return {
        "train_acc": train_acc_list,
        "val_acc": val_acc_list,
        "precision": precision_list,
        "recall": recall_list,
        "f1": f1_list,
        "cpu": cpu_usage_list,
        "gpu_mem": gpu_mem_list,
        "epoch_time": epoch_times
    }


In [79]:
def plot_metrics(metrics, scheduler_name, lr_variant, epochs):
    import matplotlib.pyplot as plt

    epochs_range = range(1, epochs+1)

    # Plot Accuracy
    plt.figure(figsize=(10,5))
    plt.plot(epochs_range, metrics['train_acc'], label='Train Accuracy')
    plt.plot(epochs_range, metrics['val_acc'], label='Validation Accuracy')
    plt.title(f'{scheduler_name} (LR: {lr_variant}) Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.show()

    # Plot Precision, Recall, F1
    plt.figure(figsize=(10,5))
    plt.plot(epochs_range, metrics['precision'], label='Precision')
    plt.plot(epochs_range, metrics['recall'], label='Recall')
    plt.plot(epochs_range, metrics['f1'], label='F1-Score')
    plt.title(f'{scheduler_name} (LR: {lr_variant}) Precision, Recall, F1')
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.legend()
    plt.show()

    # Plot CPU and GPU memory usage
    plt.figure(figsize=(10,5))
    plt.plot(epochs_range, metrics['cpu'], label='CPU Usage (%)')
    plt.plot(epochs_range, metrics['gpu_mem'], label='GPU Memory (MB)')
    plt.title(f'{scheduler_name} (LR: {lr_variant}) Resource Usage')
    plt.xlabel('Epochs')
    plt.ylabel('CPU / GPU')
    plt.legend()
    plt.show()

    # Plot Epoch time
    plt.figure(figsize=(10,5))
    plt.plot(epochs_range, metrics['epoch_time'], label='Epoch Time (s)')
    plt.title(f'{scheduler_name} (LR: {lr_variant}) Epoch Time')
    plt.xlabel('Epochs')
    plt.ylabel('Time (s)')
    plt.legend()
    plt.show()


In [ ]:
# ===== RUN TRAINING =====
metrics = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=20
)

# ===== PLOT RESULTS =====
plot_metrics(
    metrics,
   scheduler_name="CosineAnnealingLR",
    lr_variant="lr=0.001_Tmax=20",
    epochs=20
)



Epoch [1/20] Train Acc: 24.65% | Val Acc: 36.43% | Precision: 0.34 | Recall: 0.36 | F1: 0.34 | Time: 36.16s
Epoch [2/20] Train Acc: 36.81% | Val Acc: 43.40% | Precision: 0.45 | Recall: 0.43 | F1: 0.43 | Time: 35.65s
Epoch [3/20] Train Acc: 43.28% | Val Acc: 48.10% | Precision: 0.49 | Recall: 0.48 | F1: 0.47 | Time: 35.74s
Epoch [4/20] Train Acc: 48.36% | Val Acc: 53.77% | Precision: 0.55 | Recall: 0.54 | F1: 0.54 | Time: 35.50s
Epoch [5/20] Train Acc: 52.96% | Val Acc: 58.58% | Precision: 0.59 | Recall: 0.59 | F1: 0.58 | Time: 35.88s
Epoch [6/20] Train Acc: 56.66% | Val Acc: 61.97% | Precision: 0.62 | Recall: 0.62 | F1: 0.62 | Time: 36.00s
Epoch [7/20] Train Acc: 59.88% | Val Acc: 63.18% | Precision: 0.64 | Recall: 0.63 | F1: 0.63 | Time: 35.73s
Epoch [8/20] Train Acc: 62.56% | Val Acc: 65.38% | Precision: 0.65 | Recall: 0.65 | F1: 0.65 | Time: 36.07s
Epoch [9/20] Train Acc: 64.37% | Val Acc: 68.85% | Precision: 0.69 | Recall: 0.69 | F1: 0.69 | Time: 36.19s
Epoch [10/20] Train Acc: 66.